# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [44]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [31]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# import re

# # Define scope and credentials
# scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name("autoscraper-380600-0d0c84856d6b.json", scope)
# client = gspread.authorize(creds)

# # Open the Google Sheet
# sheet = client.open("BrandonMillerFORMAP").sheet1  # Adjust sheet1 if using different tabs

# # Get a range of cells (adjust range as needed)
# cell_range = sheet.range('A1:D13')  # Change to match your sheet range

# # Get the headers and the actual data range (adjust as necessary)
# headers = sheet.row_values(1)  # Fetch first row for headers
# data_range = sheet.get_all_values()[1:]  # Skip header row

# # Extract hyperlinks and clean data
# data = []

# def extract_hyperlink_info(formula):
#     """Extracts URL and display text from a Google Sheets HYPERLINK formula."""
#     if not formula:
#         return None, None  # Return None if there's no formula
#     pattern = r'HYPERLINK\("(.+?)",\s*"(.+?)"\)'
#     match = re.search(pattern, formula)
#     if match:
#         return match.group(2), match.group(1)  # return (display_text, url)
#     return None, None

# # Loop through rows and process the data
# for row in sheet.get_all_records():
#     # Extract values
#     address = row['Address']  # Adjust these keys if different
#     description = row['Description']
#     neighborhood = row['Neighborhood']

#     # Check if address or description contains hyperlinks
#     address_formula = sheet.cell(row['Row'], 1, value_render_option='FORMULA').value  # Address column
#     description_formula = sheet.cell(row['Row'], 2, value_render_option='FORMULA').value  # Description column
    
#     # Extract hyperlink from Address and Description columns (if present)
#     address_display, address_url = extract_hyperlink_info(address_formula)
#     description_display, description_url = extract_hyperlink_info(description_formula)
    
#     # Combine hyperlink into the display text
#     if address_url:
#         address = f'<a href="{address_url}">{address_display}</a>'
#     if description_url:
#         description = f'<a href="{description_url}">{description_display}</a>'
    
#     # Append the cleaned row to data
#     data.append([address, description, neighborhood])

# # Convert to DataFrame
# df = pd.DataFrame(data, columns=headers)

# # Display the dataframe to ensure data integrity
# print(df)

In [45]:
df = pd.read_csv('BrandonMillerFORMAP - Sheet1.csv')

In [46]:
def create_hyperlink_limited(description, url, word_limit=5):
    """Create an HTML anchor tag for the first few words of the description with the URL."""
    if pd.notnull(url):  # Check if there's a valid URL
        words = description.split()  # Split the description into words
        if len(words) > word_limit:  # Only wrap the first word_limit words
            first_part = ' '.join(words[:word_limit])
            remaining_part = ' '.join(words[word_limit:])
            return f'<a href="{url}">{first_part}</a> {remaining_part}'
        else:
            return f'<a href="{url}">{description}</a>'  # If fewer than word_limit words, wrap the whole text
    return description  # Return original description if no URL

# Apply the function to combine 'Description' and 'URL'
df['Description'] = df.apply(lambda row: create_hyperlink_limited(row['Description'], row['URL'], word_limit=5), axis=1)

In [47]:
# Drop the 'URL' column if you no longer need it
df = df.drop(columns=['URL'])

In [48]:
df['Description'].iloc[0]

'<a href="https://therealdeal.com/new-york/2019/11/14/new-boutique-office-building-headed-for-east-villages-st-marks/">REEC bought the leasehold in</a> 2017 for $29M from Edward Gabay, planning a 53,000-sf office building. In 2021, foundations had yet to be dug, and Madison Capital Realty moved to foreclose. Parkview Financial rescued the project with a $70M refi the next year. The 9-story building is still under construction.'

## Geocode

In [49]:
df

,Address,Description,Neighborhood
0,3 St. Marks Place,"<a href=""https://therealdeal.com/new-york/2019...",NYC
1,118 10th Avenue,"<a href=""https://therealdeal.com/new-york/2024...",NYC
2,2226 Third Avenue,"After buying the leasehold in 2020 for $27M, R...",NYC
3,313-315 West 125th Street,"<a href=""https://www.pincusco.com/reec-was-par...",NYC
4,196 Orchard Street,REEC teamed up with Magnum Real Estate Group i...,NYC
5,1228 Madison Avenue,REEC acquired the ground lease in 2017 for $15...,NYC
6,958 Madison Avenue,"<a href=""https://www.crainsnewyork.com/real-es...",NYC
7,162-166 Bowery,"<a href=""https://www.pincusco.com/reec-files-p...",NYC
8,25 Cobb Isle Road,"<a href=""https://therealdeal.com/new-york/2024...",NYC
9,137 Franklin Street,"<a href=""https://therealdeal.com/new-york/2024...",NYC


In [50]:
df['full_address'] = df['Address'] + ' ' + df['Neighborhood'] + ' NY'

In [51]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [52]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Address,Description,Neighborhood,full_address,geocoded,lat,lon,geometry
0,3 St. Marks Place,"<a href=""https://therealdeal.com/new-york/2019...",NYC,3 St. Marks Place NYC NY,"(40.7295381, -73.9893618)",40.729538,-73.989362,POINT (-73.98936 40.72954)
1,118 10th Avenue,"<a href=""https://therealdeal.com/new-york/2024...",NYC,118 10th Avenue NYC NY,"(40.7441775, -74.00628499999999)",40.744177,-74.006285,POINT (-74.00628 40.74418)
2,2226 Third Avenue,"After buying the leasehold in 2020 for $27M, R...",NYC,2226 Third Avenue NYC NY,"(40.808487, -73.9314989)",40.808487,-73.931499,POINT (-73.93150 40.80849)
3,313-315 West 125th Street,"<a href=""https://www.pincusco.com/reec-was-par...",NYC,313-315 West 125th Street NYC NY,"(40.8108017, -73.95182539999999)",40.810802,-73.951825,POINT (-73.95183 40.81080)
4,196 Orchard Street,REEC teamed up with Magnum Real Estate Group i...,NYC,196 Orchard Street NYC NY,"(40.722264, -73.987985)",40.722264,-73.987985,POINT (-73.98798 40.72226)
5,1228 Madison Avenue,REEC acquired the ground lease in 2017 for $15...,NYC,1228 Madison Avenue NYC NY,"(40.7825765, -73.9578338)",40.782576,-73.957834,POINT (-73.95783 40.78258)
6,958 Madison Avenue,"<a href=""https://www.crainsnewyork.com/real-es...",NYC,958 Madison Avenue NYC NY,"(40.7740844, -73.964131)",40.774084,-73.964131,POINT (-73.96413 40.77408)
7,162-166 Bowery,"<a href=""https://www.pincusco.com/reec-files-p...",NYC,162-166 Bowery NYC NY,"(40.7201299, -73.9944347)",40.720130,-73.994435,POINT (-73.99443 40.72013)
8,25 Cobb Isle Road,"<a href=""https://therealdeal.com/new-york/2024...",NYC,25 Cobb Isle Road NYC NY,"(40.7127753, -74.0059728)",40.712775,-74.005973,POINT (-74.00597 40.71278)
9,137 Franklin Street,"<a href=""https://therealdeal.com/new-york/2024...",NYC,137 Franklin Street NYC NY,"(40.7189195, -74.00722139999999)",40.718919,-74.007221,POINT (-74.00722 40.71892)


In [53]:
gdf

,Address,Description,Neighborhood,full_address,geocoded,lat,lon,geometry
0,3 St. Marks Place,"<a href=""https://therealdeal.com/new-york/2019...",NYC,3 St. Marks Place NYC NY,"(40.7295381, -73.9893618)",40.729538,-73.989362,POINT (-73.98936 40.72954)
1,118 10th Avenue,"<a href=""https://therealdeal.com/new-york/2024...",NYC,118 10th Avenue NYC NY,"(40.7441775, -74.00628499999999)",40.744177,-74.006285,POINT (-74.00628 40.74418)
2,2226 Third Avenue,"After buying the leasehold in 2020 for $27M, R...",NYC,2226 Third Avenue NYC NY,"(40.808487, -73.9314989)",40.808487,-73.931499,POINT (-73.93150 40.80849)
3,313-315 West 125th Street,"<a href=""https://www.pincusco.com/reec-was-par...",NYC,313-315 West 125th Street NYC NY,"(40.8108017, -73.95182539999999)",40.810802,-73.951825,POINT (-73.95183 40.81080)
4,196 Orchard Street,REEC teamed up with Magnum Real Estate Group i...,NYC,196 Orchard Street NYC NY,"(40.722264, -73.987985)",40.722264,-73.987985,POINT (-73.98798 40.72226)
5,1228 Madison Avenue,REEC acquired the ground lease in 2017 for $15...,NYC,1228 Madison Avenue NYC NY,"(40.7825765, -73.9578338)",40.782576,-73.957834,POINT (-73.95783 40.78258)
6,958 Madison Avenue,"<a href=""https://www.crainsnewyork.com/real-es...",NYC,958 Madison Avenue NYC NY,"(40.7740844, -73.964131)",40.774084,-73.964131,POINT (-73.96413 40.77408)
7,162-166 Bowery,"<a href=""https://www.pincusco.com/reec-files-p...",NYC,162-166 Bowery NYC NY,"(40.7201299, -73.9944347)",40.720130,-73.994435,POINT (-73.99443 40.72013)
8,25 Cobb Isle Road,"<a href=""https://therealdeal.com/new-york/2024...",NYC,25 Cobb Isle Road NYC NY,"(40.7127753, -74.0059728)",40.712775,-74.005973,POINT (-74.00597 40.71278)
9,137 Franklin Street,"<a href=""https://therealdeal.com/new-york/2024...",NYC,137 Franklin Street NYC NY,"(40.7189195, -74.00722139999999)",40.718919,-74.007221,POINT (-74.00722 40.71892)


In [54]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [55]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [56]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NYC_Brandon_Miller_09_20_24


<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>

In [57]:
gdf.columns

Index(['Address', 'Description', 'Neighborhood', 'full_address', 'geocoded',
       'lat', 'lon', 'geometry'],
      dtype='object')